In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import FaceDetectors
import random
from PIL import Image, ImageSequence 

# --- 1. FUNCIÓN PARA CARGAR GIF ---
def cargar_gif_como_lista(ruta_gif):
    frames_list = []
    pil_image = Image.open(ruta_gif)
    for frame in ImageSequence.Iterator(pil_image):
        frame = frame.convert("RGBA")
        opencv_image = np.array(frame)
        opencv_image = cv2.cvtColor(opencv_image, cv2.COLOR_RGBA2BGRA)
        frames_list.append(opencv_image)
    print(f"✅ GIF cargado: {ruta_gif} ({len(frames_list)} frames)")
 
    return frames_list

# --- 2. FUNCIÓN DE MEZCLA ---
def overlay_transparent(background, overlay, x, y, overlay_size=None):
    bg_h, bg_w, _ = background.shape
    if overlay_size is not None:
        overlay = cv2.resize(overlay.copy(), overlay_size)
    img_h, img_w, c = overlay.shape
    
    if c == 3: overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2BGRA)

    if x + img_w > bg_w: img_w = bg_w - x
    if y + img_h > bg_h: img_h = bg_h - y
    if x < 0: img_w = img_w + x; x = 0
    if y < 0: img_h = img_h + y; y = 0
    if img_w <= 0 or img_h <= 0: return background

    crop_fg = overlay[:img_h, :img_w]
    crop_bg = background[y:y+img_h, x:x+img_w]
    
    alpha = crop_fg[:, :, 3] / 255.0
    alpha_inv = 1.0 - alpha
    
    for c in range(0, 3):
        crop_bg[:, :, c] = (alpha * crop_fg[:, :, c] + alpha_inv * crop_bg[:, :, c])
    return background

# --- 3. CARGAR ASSETS ---
print(" Cargando modelo IA y Gráficos...")
classifier = load_model('modelo_emociones.h5')
labels = ['Enojado', 'Asco', 'Miedo', 'Feliz', 'Neutral', 'Triste', 'Sorpresa']

# IMAGEN ESTÁTICA (Feliz)
img_feliz = cv2.imread('img_feliz.png', cv2.IMREAD_UNCHANGED)
if img_feliz is None: 
    img_feliz = np.zeros((100, 100, 4), dtype=np.uint8); img_feliz[:] = (0, 255, 0, 255)

# GIF ANIMADO (Enfado)
gif_fuego_frames = cargar_gif_como_lista('fuego.gif')
idx_fuego = 0 

# --- 4. CONFIGURACIÓN ---
FDet = FaceDetectors.FaceDetector()
cap = cv2.VideoCapture(0)
cap.set(3, 640); cap.set(4, 480)
font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    ret, frame = cap.read()
    if not ret: break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detectar cara
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
    
    if values and values[0] is not None:
        face, eyes, shape = values
        (x, y, w, h) = face 
        
        # --- PREDICCIÓN ---
        try:
            roi_gray = gray[y:y+h, x:x+w]
            
            if roi_gray.size > 0:
                roi = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
                roi = roi.astype("float") / 255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi, axis=0)

                preds = classifier.predict(roi, verbose=0)[0]
                label = labels[preds.argmax()]
                
                # --- LÓGICA DE FILTROS ---
                
                # CASO 1: FELIZ (Imagen Estática + Confeti)
                if label == 'Feliz':
                    # 1. Imagen Estática
                    ancho_gorro = int(w * 1.0)
                    factor = ancho_gorro / img_feliz.shape[1]
                    alto_gorro = int(img_feliz.shape[0] * factor)
                    pos_x = x + int(w/2) - int(ancho_gorro/2)
                    pos_y = y - int(alto_gorro * 0.9)
                    
                    try: frame = overlay_transparent(frame, img_feliz, pos_x, pos_y, (ancho_gorro, alto_gorro))
                    except: pass
                    
                    # 2. Confeti
                    for _ in range(5):
                        cx = x + np.random.randint(-50, w+50)
                        cy = y + np.random.randint(-50, h+50)
                        color = (np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255))
                        cv2.circle(frame, (cx, cy), np.random.randint(3, 8), color, -1)

                    cv2.putText(frame, "FELICIDAD!", (x, y+h+40), font, 1, (0, 255, 0), 2)

                # CASO 2: ENOJADO (GIF Animado + Ojos Rojos)
                elif label == 'Enojado':
                    # 1. GIF Animado (Fuego)
                    current_frame = gif_fuego_frames[idx_fuego] # Coger frame actual
                    
                    ancho_fuego = int(w * 1.5)
                    factor = ancho_fuego / current_frame.shape[1]
                    alto_fuego = int(current_frame.shape[0] * factor)
                    pos_x = x + int(w/2) - int(ancho_fuego/2)
                    pos_y = y - int(alto_fuego)
                    
                    try: frame = overlay_transparent(frame, current_frame, pos_x, pos_y, (ancho_fuego, alto_fuego))
                    except: pass
                    
                    # Avanzar animación
                    idx_fuego = (idx_fuego + 1) % len(gif_fuego_frames)
                    
                    # 2. Ojos Rojos
                    [lex, ley, rex, rey] = eyes
                    if lex > 0 and rex > 0:
                        cv2.circle(frame, (int(lex), int(ley)), 8, (0, 0, 255), -1)
                        cv2.circle(frame, (int(rex), int(rey)), 8, (0, 0, 255), -1)
                        cv2.circle(frame, (int(lex), int(ley)), 2, (255, 255, 255), -1)
                        cv2.circle(frame, (int(rex), int(rey)), 2, (255, 255, 255), -1)

                    cv2.putText(frame, "GRRRRR!!!", (x, y+h+40), font, 1, (0, 0, 255), 2)

                # CASO 3: RESTO
                else:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 1)
                    cv2.putText(frame, label, (x, y-10), font, 0.8, (255, 255, 255), 2)

        except Exception as e:
            print(e)

    cv2.imshow('Filtros Emocionales', frame)
    if cv2.waitKey(20) == 27: break

cap.release()
cv2.destroyAllWindows()

 Cargando modelo IA y Gráficos...


✅ GIF cargado: fuego.gif (104 frames)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)
DetectLargestFaceEyesDLIB (480, 640, 3)
Eyes (480, 640, 3)